# YOLO Training in Google Colab

This notebook demonstrates how to train the YOLO model using Google Colab's GPU.

In [ ]:
# Clone repository
!git clone https://github.com/yourusername/yolo-pytorch.git
%cd yolo-pytorch

# Install requirements
!pip install -r requirements.txt

# Add to Python path
import sys
sys.path.append('/content/yolo-pytorch')

In [ ]:
# Download VOC dataset (adjust URL as needed)
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
!tar -xvf VOCtrainval_11-May-2012.tar

# Create dataset directories
!mkdir -p data/raw data/processed/train data/processed/val data/weights

In [ ]:
from src.scripts.convert_voc_to_yolo import main as convert_dataset
convert_dataset()

In [ ]:
from src.utils.train import main as train_model
train_model()

In [ ]:
!pytest tests/

In [ ]:
from src.utils.predict import YOLOPredictor

# Initialize predictor
predictor = YOLOPredictor(
    model_path='data/weights/best_model.pth',
    conf_threshold=0.5,
    nms_threshold=0.4
)

# Predict on an image
image_path = 'path/to/your/test/image.jpg'
detections, annotated_image = predictor.predict(image_path)

# Display results
from matplotlib import pyplot as plt
plt.imshow(annotated_image)
plt.axis('off')
plt.show()

In [ ]:
# Commit and push changes to GitHub
!git add .
!git commit -m "Update model weights and training results"
!git push origin main